In [1]:
import pandas as pd


In [6]:
labels = pd.read_csv('./data/keyword_label.csv', dtype=str)

In [7]:
labels

,keyword,label
0,골든차일드,0
1,라푼젤,1
2,콜렉트북,2
3,녕터리,3
4,마마무,4
...,...,...
101240,까르띠에산토스,101240
101241,케이스티파이 맥스,101241
101242,케이스티파이 mac,101242
101243,라마다,101243


In [9]:
pd.read_csv('./data/fm_libsvm.csv')

,?column?
0,4 27113:1 101244:1
1,15 27097:1 101244:1
2,3 191:1 101244:1
3,4 51870:1 101244:1
4,7 25523:1 101244:1
...,...
366275,4 26735:1 203792:1
366276,3 93578:1 203794:1
366277,4 25554:1 203798:1
366278,3 97676:1 203801:1


In [8]:
pd.read_csv('rec_rfm_libsvm.csv')

,4 1:1 4028794:1 4585691:1
0,1 7507:1 4028797:1 4585410:1
1,1 5:1 4028800:1 4585696:1
2,1 7511:1 4028803:1 4585699:1
3,1 8:1 3770510:1 4585700:1
4,1 10:1 4028808:1 4585420:1
...,...
26840683,1 3533334:1 4082298:1 4585415:1
26840684,1 3741771:1 3884445:1 4585796:1
26840685,1 3753835:1 4041188:1 4585763:1
26840686,2 3770404:1 4162033:1 4585753:1


In [11]:
import psycopg2

n = 0.6  # min confidence; 1/search_cnt^n
min_lift = 1
confidence_weight = 0.7
lift_weight = 0.3

query = f"""
DROP TABLE IF EXISTS workspace.kyle_search_term_log_modified;
CREATE TABLE workspace.kyle_search_term_log_modified AS (
    SELECT user_id,                                                                                                        --uid
           session_id,                                                                                                     --세션 id
           lower(trim(regexp_replace(regexp_replace(search_term, '\\r|\\n', ' '), '\\s+', ' '))) AS keyword,               --검색어
           LAG(keyword, 1)
           OVER ( PARTITION BY user_id
               ORDER BY server_time_kst)                                                         AS prev_keyword,          --이전 검색어
           LEAD(keyword, 1)
           OVER ( PARTITION BY user_id
               ORDER BY server_time_kst)                                                         AS next_keyword,          --다음 검색어
           REPLACE(search_term, ' ', '')                                                         AS keyword_no_space,      --검색어 띄어쓰기 없애기
           REPLACE(prev_keyword, ' ', '')                                                        AS prev_keyword_no_space, --이전 검색어 띄어쓰기 없애기
           REPLACE(next_keyword, ' ', '')                                                        AS next_keyword_no_space, --다음 검색어 띄어쓰기 없애기
           server_time_kst::TIMESTAMP                                                            AS new_server_time,       --검색 시간
           LAG(server_time_kst, 1) OVER (
               PARTITION BY user_id
               ORDER BY server_time_kst)::TIMESTAMP                                              AS prev_server_time,      --이전 검색어 검색 시간
           LEAD(server_time_kst, 1) OVER (
               PARTITION BY user_id
               ORDER BY server_time_kst)::TIMESTAMP                                              AS next_server_time,      --다음 검색어 검색 시간
           datediff(SECOND, prev_server_time, new_server_time)                                   AS cur_prev_duration,     --검색 시간 차 (현재-이전)
           datediff(SECOND, prev_server_time, next_server_time)                                  AS prev_next_duration     --검색 시간 차 (다음-이전)
    FROM bun_log_db.app_event_type_search
    WHERE year || month || day >= to_char(CURRENT_DATE - INTERVAL '5 day', 'yyyymmdd')
      AND year || month || day < to_char(CURRENT_DATE, 'yyyymmdd')
      AND search_term IS NOT NULL
      AND search_term != ''
      AND search_term != ' '
      AND user_id > 0
      AND device_type IN ('a', 'i')
);
DROP TABLE IF EXISTS workspace.kyle_keyword_view_count;
CREATE TABLE workspace.kyle_keyword_view_count AS
SELECT lower(trim(regexp_replace(regexp_replace(ref_term, '\\r|\\n', ' '), '\\s+', ' '))) AS keyword,
       COUNT(*)                                                                           AS view_count
FROM bun_log_db.app_event_type_view v
WHERE year || month || day >= to_char(CURRENT_DATE - INTERVAL '5 day', 'yyyymmdd')
  AND year || month || day < to_char(CURRENT_DATE, 'yyyymmdd')
  AND event_action = 'view_content'
  AND content_type = 'product'
  AND v.ref_term IS NOT NULL
  AND v.ref_term != ''
  AND v.ref_term != ' '
  AND device_type IN ('a', 'i')
GROUP BY 1;
DROP TABLE IF EXISTS workspace.kyle_search_results_count;
CREATE TABLE workspace.kyle_search_results_count AS
SELECT lower(trim(regexp_replace(regexp_replace(search_term, '\\r|\\n', ' '), '\\s+', ' '))) AS keyword,
       MAX((search_results_count):: int)                                                     AS results_count
FROM bun_log_db.api_event_type_search
WHERE year || month || day >= to_char(CURRENT_DATE - INTERVAL '5 day', 'yyyymmdd')
  AND year || month || day < to_char(CURRENT_DATE, 'yyyymmdd')
  AND device_type IN ('a', 'i')
GROUP BY 1;
DROP TABLE IF EXISTS workspace.kyle_keyword_search_count;
CREATE TABLE workspace.kyle_keyword_search_count AS
SELECT keyword,
       count(*) AS search_count
FROM workspace.kyle_search_term_log_modified
GROUP BY 1;
DROP TABLE IF EXISTS workspace.kyle_related_keyword_prev_cur;
CREATE TABLE workspace.kyle_related_keyword_prev_cur AS
SELECT prev_keyword AS keyword,
       keyword      AS related_keyword,
       COUNT(*)     AS COUNT
FROM workspace.kyle_search_term_log_modified
WHERE prev_keyword IS NOT NULL
  AND cur_prev_duration <= 600
  AND cur_prev_duration >= 1
  AND keyword_no_space != prev_keyword_no_space
GROUP BY 1, 2;
DROP TABLE IF EXISTS workspace.kyle_related_keyword_prev_next;
CREATE TABLE workspace.kyle_related_keyword_prev_next AS
SELECT prev_keyword AS keyword,
       next_keyword AS related_keyword,
       COUNT(*)     AS COUNT
FROM workspace.kyle_search_term_log_modified
WHERE prev_keyword IS NOT NULL
  AND cur_prev_duration <= 300
  AND cur_prev_duration >= 1
  AND next_keyword_no_space != prev_keyword_no_space
GROUP BY 1, 2;
DROP TABLE IF EXISTS workspace.kyle_related_keyword_combined;
CREATE TABLE workspace.kyle_related_keyword_combined AS
SELECT *
FROM (SELECT c.keyword,
             c.related_keyword,
             c.count + n.count                                             AS related_count,
             (SELECT COUNT(*)
              FROM kyle_search_term_log_modified)                          AS total_search_count,
             sc.search_count,
             rsc.search_count                                              AS related_search_count,
             related_count / total_search_count :: float                   AS support,
             related_count / sc.search_count :: float                      AS confidence,
             confidence / (rsc.search_count / total_search_count :: float) AS lift
      FROM workspace.kyle_related_keyword_prev_cur c
               JOIN workspace.kyle_related_keyword_prev_next n ON c.keyword = n.keyword
          AND c.related_keyword = n.related_keyword
               JOIN workspace.kyle_keyword_search_count sc ON c.keyword = sc.keyword
               JOIN workspace.kyle_keyword_search_count rsc ON c.related_keyword = rsc.keyword) a
WHERE related_count > 2;
DROP TABLE IF EXISTS workspace.kyle_related_keyword_df_temp;
CREATE TABLE workspace.kyle_related_keyword_df_temp AS
SELECT k.keyword,
       k.related_keyword,
       k.related_count,
       k.search_count,
       k.related_search_count,
       rc.results_count,
       v.view_count AS related_view_count,
       k.total_search_count,
       k.support,
       k.confidence,
       k.lift
FROM workspace.kyle_related_keyword_combined k
         JOIN workspace.kyle_search_results_count rc ON k.related_keyword = rc.keyword
         JOIN workspace.kyle_keyword_view_count v ON k.related_keyword = v.keyword
WHERE results_count > 1
ORDER BY 4 DESC, 1, 10 DESC;
DROP TABLE IF EXISTS workspace.kyle_related_keyword_df_filtered;
CREATE TABLE workspace.kyle_related_keyword_df_filtered AS
SELECT *
FROM workspace.kyle_related_keyword_df_temp
WHERE confidence > 1 / pow(search_count, 0.6)
  AND lift > 1;
DROP TABLE IF EXISTS workspace.kyle_related_keyword_df_max;
CREATE TABLE workspace.kyle_related_keyword_df_max AS
SELECT keyword,
       max(confidence) AS max_confidence,
       max(lift)       AS max_lift
FROM workspace.kyle_related_keyword_df_filtered
GROUP BY 1;
SELECT keyword,
       related_keyword,
       related_count,
       search_count,
       related_search_count,
       results_count,
       related_view_count,
       total_search_count,
       support,
       confidence,
       lift,
       score
FROM (SELECT a.*,
       ROUND((0.7 * a.confidence / b.max_confidence:: float) +
             (0.3 * a.lift / b.max_lift:: float), 4) AS score,
       row_number() over(
           PARTITION BY a.keyword
           ORDER BY score DESC
           ) AS rank
FROM workspace.kyle_related_keyword_df_filtered a
         JOIN workspace.kyle_related_keyword_df_max b ON a.keyword = b.keyword) c
ORDER BY search_count DESC, keyword, score DESC;
"""
df = pd.read_sql(query, conn)
conn.close()

In [12]:
df

,keyword,related_keyword,related_count,search_count,related_search_count,results_count,related_view_count,total_search_count,support,confidence,lift,score
0,자전거,로드자전거,368,17308,4815,1070,23292,10242675,3.592811e-05,0.021262,4.522911e+01,0.8029
1,자전거,mtb자전거,239,17308,2652,1154,10888,10242675,2.333375e-05,0.013809,5.333237e+01,0.5759
2,자전거,중고자전거,78,17308,350,123,1344,10242675,7.615198e-06,0.004507,1.318843e+02,0.4484
3,자전거,전기자전거,174,17308,3947,1185,14181,10242675,1.698775e-05,0.010053,2.608847e+01,0.3903
4,자전거,중고전기자전거,111,17308,934,14,989,10242675,1.083701e-05,0.006413,7.033032e+01,0.3711
...,...,...,...,...,...,...,...,...,...,...,...,...
101793,힌정판 샤프,한정판 샤프,3,2,10,93,30,10242675,2.928922e-07,1.500000,1.536401e+06,1.0000
101794,힐스 id,힐스,3,2,22,392,34,10242675,2.928922e-07,1.500000,6.983642e+05,1.0000
101795,힐일버드,힐링버드,3,2,37,30,108,10242675,2.928922e-07,1.500000,4.152436e+05,1.0000
101796,힐튼이불,라지킹침대,3,2,8,27,23,10242675,2.928922e-07,1.500000,1.920502e+06,1.0000


In [13]:
df.to_csv('df.csv', index=False)

In [16]:
for i in df.keyword.unique():
    if '아이폰' in i:
        print(i)

아이폰
아이폰x
아이폰12
아이폰11
아이폰8
아이폰12미니
아이폰7
아이폰xr
아이폰 e2
아이폰 e
아이폰6
아이폰 x
아이폰11프로
아이폰12프로
아이폰 11
아이폰  e2
아이폰11pro
아이폰 12
아이폰12 미니
아이폰x  max
아이폰 8
아이폰12프로맥스
아이폰 xr
아이폰8플러스
아이폰5
아이폰 e1
아이폰12pro
아이폰  e
아이폰 미니
아이폰x  256
아이폰 7
아이폰 6
아이폰미니
아이폰 12 미니
아이폰 충전기
아이폰11프로맥스
아이폰 12미니
아이폰 공기계
아이폰8+
아이폰7플러스
아이폰4
아이폰 11pro
아이폰 케이스
아이폰케이스
아이폰12 프로
아이폰11 128
아이폰 이어폰
아이폰x  64
아이폰x  케이스
아이폰 e2 128
아이폰 미개봉
아이폰pro max
아이폰 교환
아이폰 e  스페이스그레이
아이폰11 프로
아이폰 5
아이폰 11프로
아이폰미니12
아이폰12 프로맥스
아이폰11 pro
아이폰 11 pro
아이폰10
아이폰xr 128
대구 아이폰
아이폰 교신
아이폰12 화이트
아이폰 x  max
아이폰12 pro
아이폰8 플러스
아이폰 11 프로
아이폰12 pro max
아이폰 12프로
아이폰  e1
아이폰12 교환
아이폰미니 케이스
아이폰프로맥스12
아이폰11 퍼플
아이폰12미니케이스
아이폰 12pro
아이폰프로맥스
아이폰 8+
아이폰 8플러스
아이폰7 128
아이폰 무음
아이폰 프로맥스
아이폰x max
아이폰 부품용
아이폰7+
아이폰xr 64
아이폰 8 플러스
아이폰 e2 미개봉
아이폰6 플러스
아이폰pro
아이폰12 그린
아이폰 부품
아이폰12mini
아이폰 e2 64
아이폰xa
아이폰x  max 256
아이폰12프로 교환
아이폰9
광주 아이폰
아이폰8 레드
아이폰프로맥스256
아이폰11 256
아이폰12 64
아이폰12 정품 케이스
아이폰xs
아이폰교환
아이폰 e2 케이스
아이폰플러스
아이폰11pro 256
아이폰 파손
아이폰x  골드
부산 아이폰
아이폰6플러스
아이폰12 미니 케이스
아이폰12미니 교환
아이폰교

In [22]:
category_label = pd.read_csv('rec_rfm_category_label.csv000.gz', dtype=str, header=None, compression='gzip')

In [21]:
keyword_label = pd.read_csv('./data/keyword_label.csv', dtype=str)

In [23]:
category_label.shape

(1046, 2)

In [24]:
keyword_label.shape

(1822674, 2)

In [25]:
category_label.iloc[:,1]

0       1035
1        938
2        855
3        874
4         55
        ... 
1041     617
1042     633
1043     858
1044     244
1045       0
Name: 1, Length: 1046, dtype: object

In [26]:
keyword_label.iloc[:, 1]

0           948942
1           255263
2          1576964
3          1697959
4          1445811
            ...   
1822669    1342958
1822670     374847
1822671    1101805
1822672    1805924
1822673     490210
Name: label, Length: 1822674, dtype: object